In [3]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
# Load tumor images
tumor = []
tumor_path = 'C:\\Users\\lenovo\\Downloads\\Brain_tumour_dataset\\yes/*.jpg'
for f in glob.glob(tumor_path):
    img = cv2.imread(f)
    img = cv2.resize(img, (128, 128))  # Resize to 128x128
    img = img / 255.0  # Normalize pixel values
    tumor.append(img)

# Load healthy images
healthy = []
healthy_path = 'C:\\Users\\lenovo\\Downloads\\Brain_tumour_dataset\\no/*.jpg'
for f in glob.glob(healthy_path):
    img = cv2.imread(f)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    healthy.append(img)

# Convert lists to NumPy arrays
tumor = np.array(tumor)
healthy = np.array(healthy)

# Create labels (1 for tumor, 0 for healthy)
tumor_labels = np.ones(len(tumor))
healthy_labels = np.zeros(len(healthy))

# Combine data
X = np.concatenate((tumor, healthy), axis=0)
y = np.concatenate((tumor_labels, healthy_labels), axis=0)

# One-hot encoding labels
y = to_categorical(y, 2)

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (2856, 128, 128, 3)
Testing set shape: (714, 128, 128, 3)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes (healthy & tumor)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of model
model.summary()


c:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 34s 344ms/step - accuracy: 0.7997 - loss: 0.5003 - val_accuracy: 0.9538 - val_loss: 0.1548
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 174ms/step - accuracy: 0.9435 - loss: 0.1539 - val_accuracy: 0.9594 - val_loss: 0.1209
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 173ms/step - accuracy: 0.9532 - loss: 0.1222 - val_accuracy: 0.9454 - val_loss: 0.1466
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 173ms/step - accuracy: 0.9650 - loss: 0.0860 - val_accuracy: 0.9664 - val_loss: 0.0959
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 172ms/step - accuracy: 0.9784 - loss: 0.0571 - val_accuracy: 0.9608 - val_loss: 0.1163
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 175ms/step - accuracy: 0.9763 - loss: 0.0556 - val_accuracy: 0.9706 - val_loss: 0.1448
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 172ms/step - accuracy: 0.9824 - loss: 0.0390 - val_accuracy: 0.9706 - val_loss: 0.1093
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 16s 176ms/step - accuracy: 0.9872 - loss: 0.0333 - val_accu

In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.9693 - loss: 0.1343
Test Accuracy: 96.92%


In [8]:
import cv2
import numpy as np
import os

def predict_image(img_path):
    # Ensure the file exists
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"File not found: {img_path}")

    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Failed to load the image. Check the file path and format.")

    # Resize the image
    img = cv2.resize(img, (128, 128))
    
    # Normalize
    img = img / 255.0
    
    # Add batch dimension
    img = np.expand_dims(img, axis=0)

    return img  # Return preprocessed image for model prediction

# Example usage:
img_path =  r"C:\Users\lenovo\Downloads\Brain_tumour_dataset\yes\ct_tumor (1).jpg"

img = predict_image(img_path)

# Now use your trained model to predict
prediction = model.predict(img)
print("Prediction:", prediction)
print("Predicted class:", np.argmax(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Prediction: [[0.98837566 0.01162432]]
Predicted class: 0


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
model.evaluate(X_test, y_test)



23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9693 - loss: 0.1343


[0.10972864180803299, 0.9691876769065857]

In [11]:
model.save("brain_tumor_classifier.keras")


In [ ]:
# Install tensorflow package
%pip install tensorflow

from tensorflow.keras.models import load_model

model = load_model("brain_tumor_classifier.keras")  # Load the saved model
